In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## SurrealDB

[SurrealDB](https://surrealdb.com/) has emerged as a multi-mode performant data-base management system that also allows for vector search. Agno offers support to this type of data base: https://docs.agno.com/vectordb/surrealdb

You might select between Cloud hosted of self-hosted options:
```sh
docker run --rm \
  --pull always \
  -p 8000:8000 \
  surrealdb/surrealdb:latest \
  start \
  --user root \
  --pass root
```

Either way, a **namespace**, **database**, **user** and **password** must be provided.

In [2]:
import os
from surrealdb import Surreal

# SurrealDB connection parameters
SURREALDB_URL = os.getenv("SURREAL_HOST")
SURREALDB_USER = os.getenv("SURREAL_USER")
SURREALDB_PASSWORD = os.getenv("SURREAL_PASS")
SURREALDB_NAMESPACE = os.getenv("SURREAL_NAMESPACE")
SURREALDB_DATABASE = os.getenv("SURREAL_DB")

# Create a client
client = Surreal(url=SURREALDB_URL)
client.signin({"username": SURREALDB_USER, "password": SURREALDB_PASSWORD})
client.use(namespace=SURREALDB_NAMESPACE, database=SURREALDB_DATABASE)

Given the multi-model versatile schema that SurrealDB provides we can do complex searches that take into consideration semantical closeness as well as relational graph between chunk. The **Hierarchical Navigable Small World** (HNSW) algorithm is a graph-based approximate nearest neighbor search technique used in many vector databases. Nearest neighbor search without an index involves computing the distance from the query to each point in the database, which for large datasets is computationally prohibitive.

https://www.pinecone.io/learn/series/faiss/hnsw/

In [6]:
from agno.vectordb.surrealdb import SurrealDb
from agno.embedder.google import GeminiEmbedder
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase

# Database
surrealdb = SurrealDb(
    client=client,
    collection="recipes",  # Collection name for storing documents
    efc=150,  # HNSW construction time/accuracy trade-off
    m=12,  # HNSW max number of connections per element
    search_ef=40,  # HNSW search time/accuracy trade-off
    embedder=GeminiEmbedder(),
)

# Knowledge base
knowledge_base = PDFUrlKnowledgeBase(
    urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
    vector_db=surrealdb
)
knowledge_base.load(recreate=True)

INFO Dropping collection

INFO Creating collection

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO Added 14 documents to knowledge base

Once loaded we can explore the embeddings and texts extracted through their UI.

![](../images/surrealdb.png)

Create an agent and ask about some Thai recipes.

In [8]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create agent and query synchronously
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    knowledge=knowledge_base,
    show_tool_calls=True
)
agent.print_response(
    "What are the 3 categories of Thai SELECT is given to restaurants overseas?",
    markdown=True,
    stream=True
)

Output()